In [1]:
import numpy as np
import soundfile as sf
from scipy.signal import stft, istft
import IPython.display as ipd

In [2]:
class CustomEqualizer:
    def __init__(self, sr, bands, n_fft=4096, hop=1024):
        """
        Args:
            sr: sample rate
            bands: list of (center_freq, gain_db, bandwidth_hz)
            n_fft, hop: STFT parameters
        """
        self.sr = sr
        self.bands = bands
        self.n_fft = n_fft
        self.hop = hop

    def apply(self, audio):
        """Apply equalizer to mono signal."""
        f, t, Z = stft(audio, fs=self.sr, nperseg=self.n_fft, noverlap=self.n_fft - self.hop)
        gain_curve = np.ones_like(f)
        for center, gain_db, bw in self.bands:
            gain = 10 ** (gain_db / 20.0)
            weight = np.exp(-0.5 * ((f - center) / (bw / 2)) ** 2)
            gain_curve *= 1 + (gain - 1) * weight
        Z_eq = Z * gain_curve[:, None]
        _, y = istft(Z_eq, fs=self.sr, nperseg=self.n_fft, noverlap=self.n_fft - self.hop)
        return y

    def update_band(self, idx, gain_db=None, center=None, bandwidth=None):
        """Update a band’s parameters easily."""
        center0, gain0, bw0 = self.bands[idx]
        self.bands[idx] = (
            center if center is not None else center0,
            gain_db if gain_db is not None else gain0,
            bandwidth if bandwidth is not None else bw0
        )

    def print_bands(self):
        print("Current Equalizer Bands:")
        for i, (c, g, bw) in enumerate(self.bands):
            print(f"[{i}] Center={c:>6.0f} Hz | Gain={g:>6.1f} dB | BW={bw:>5.0f} Hz")


In [3]:
# Change this to your actual file path
audio_path = "/Users/reiner/Documents/GitHub/cadenza_2026_submission/project/dataset/cadenza_data/train/unprocessed/6cc97125bb708b1fa5a55e7e_unproc.flac"

x, sr = sf.read(audio_path)
if x.ndim == 2:
    x = x.mean(axis=1)

print(f"Loaded audio: {len(x)/sr:.2f} seconds, sample rate: {sr} Hz")

# ▶️ Play the original audio
ipd.Audio(x, rate=sr)


Loaded audio: 4.30 seconds, sample rate: 44100 Hz


In [4]:
# Define your custom bands (center frequency, gain dB, bandwidth Hz)
# Cut the mids and boost the highs
bands = [
    (20,    -49.02,   5),
    (25,    -199.78,   6),
    (31.5,  -44.38,   7),
    (40,    -38.93,   9),
    (50,    -33.06,  12),
    (63,    -35.48,  15),
    (80,    -37.09,  19),
    (100,   -35.48,  23),
    (125,   -26.59,  29),
    (160,   -17.43,  37),
    (200,   -15.53,  46),
    (250,   -20.83,  58),
    (315,   -4.0,  73),
    (400,   -3.57,  93),
    (500,   0.0, 116),
    (630,   -6.02, 146),
    (800,   -15.50, 185),
    (1000,  -11.89, 232),
    (1250,  -21.15, 289),
    (1600,  -20.66, 371),
    (2000,  -29.67, 463),
    (2500,  -40.91, 579),
    (3150,  -63.78, 729),
    (4000,  -80.10, 926),
    (5000,  -80.08, 1158),
    (6300,  -80.27, 1459),
    (8000,  -80.51, 1853),
    (10000, -80.57, 2316),
    (12500, -80.59, 2895),
    (16000, -80.36, 3705),
    (20000, -80.64, 4631),
]

eq = CustomEqualizer(sr, bands)
eq.print_bands()

# Apply EQ
y = eq.apply(x)

# Save for reference
sf.write("equalized_output.flac", y, sr)
print("Saved as equalized_output.flac")


Current Equalizer Bands:
[0] Center=    20 Hz | Gain= -49.0 dB | BW=    5 Hz
[1] Center=    25 Hz | Gain=-199.8 dB | BW=    6 Hz
[2] Center=    32 Hz | Gain= -44.4 dB | BW=    7 Hz
[3] Center=    40 Hz | Gain= -38.9 dB | BW=    9 Hz
[4] Center=    50 Hz | Gain= -33.1 dB | BW=   12 Hz
[5] Center=    63 Hz | Gain= -35.5 dB | BW=   15 Hz
[6] Center=    80 Hz | Gain= -37.1 dB | BW=   19 Hz
[7] Center=   100 Hz | Gain= -35.5 dB | BW=   23 Hz
[8] Center=   125 Hz | Gain= -26.6 dB | BW=   29 Hz
[9] Center=   160 Hz | Gain= -17.4 dB | BW=   37 Hz
[10] Center=   200 Hz | Gain= -15.5 dB | BW=   46 Hz
[11] Center=   250 Hz | Gain= -20.8 dB | BW=   58 Hz
[12] Center=   315 Hz | Gain=  -4.0 dB | BW=   73 Hz
[13] Center=   400 Hz | Gain=  -3.6 dB | BW=   93 Hz
[14] Center=   500 Hz | Gain=   0.0 dB | BW=  116 Hz
[15] Center=   630 Hz | Gain=  -6.0 dB | BW=  146 Hz
[16] Center=   800 Hz | Gain= -15.5 dB | BW=  185 Hz
[17] Center=  1000 Hz | Gain= -11.9 dB | BW=  232 Hz
[18] Center=  1250 Hz | Gain= -

In [5]:
ipd.Audio(y, rate=sr)


In [6]:
# Change this to your actual file path
audio_path = "/Users/reiner/Documents/GitHub/cadenza_2026_submission/project/dataset/cadenza_data/train/signals/6cc97125bb708b1fa5a55e7e.flac"

x, sr = sf.read(audio_path)
if x.ndim == 2:
    x = x.mean(axis=1)

print(f"Loaded audio: {len(x)/sr:.2f} seconds, sample rate: {sr} Hz")

# ▶️ Play the original audio
ipd.Audio(x, rate=sr)


Loaded audio: 4.31 seconds, sample rate: 44100 Hz
